Dependencies

In [7]:
import pandas as pd
import numpy as np
import random
from random import normalvariate
import plotly.express as px

from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment
from cadCAD.engine import ExecutionContext, Executor

In [8]:
#State variables
initial_state = {
    'fc_minted' : 6250,
    'tree_planted': 1000,
    'fc_price': 0.5,
    'usd_balance': 2500,
    'fc_balance': 5000
}

In [9]:
#System parameters
system_params = {
    'reward_per_tree': [12.5],
}

In [10]:
#Policy functions

def p_mint_fc(params, substep, state_history, previous_state):
    r = params['reward_per_tree']
    
    #random number of trees planted
    tree = random.randint(0, 100)

    #issue coins
    new_coins = tree*r
    
    return {'add_coins': new_coins}

def p_buy_fc(params, substep, state_history, previous_state):
    
    #random usd amount to purchase fc coin
    usd_amount = random.uniform(10, 1000)
    current_price = previous_state['fc_price']

    fc_amount = -(usd_amount / current_price)

    return {'fc_trade': fc_amount, 'usd_value' : usd_amount }

def p_sell_fc(params, substep, state_history, previous_state):
    
    #random fc amount sold 
    fc_amount = random.uniform(10, 1000)
    current_price = previous_state['fc_price']

    usd_amount =  -(fc_amount * current_price)

    return {'fc_trade': fc_amount, 'usd_value' : usd_amount}

In [11]:
#State Update functions

def s_fc_minted(params, substep, state_history, previous_state, policy_input):
    current_coins = previous_state['fc_minted']
    new_coins = policy_input['add_coins']

    fc_supply = current_coins + new_coins

    return ('fc_minted', fc_supply)

def s_tree_planted(params, substep, state_history, previous_state, policy_input):
    current_trees = previous_state['tree_planted']
    r = params['reward_per_tree']

    new_trees = policy_input['add_coins'] / r

    trees_total = current_trees + new_trees

    return ('tree_planted', trees_total)

def s_fc_balance(params, substep, state_history, previous_state, policy_input):
    current_fc = previous_state['fc_balance']
    fc_change = policy_input['fc_trade']

    new_fc = current_fc + fc_change
    
    return ('fc_balance', new_fc)

def s_usd_balance(params, substep, state_history, previous_state, policy_input):
    current_usd = previous_state['usd_balance']
    usd_change = policy_input['usd_value']

    new_usd = current_usd + usd_change
    
    return ('usd_balance', new_usd)
    
def s_fc_price(params, substep, state_history, previous_state, policy_input):
    usd_balance = previous_state['usd_balance']
    fc_balance =  previous_state['fc_balance']

    new_price = usd_balance / fc_balance
    

    return ('fc_price', new_price)

In [12]:
#Partial State Update Block

partial_state_update_blocks = [
    {
        'label': 'Minting & Tree growth', # Useful metadata to describe our partial state update blocks
        'policies': {
            'add_coins': p_mint_fc,
        },
        'variables': {
            'fc_minted': s_fc_minted,
            'tree_planted': s_tree_planted    
        }
    },
    {
        'label': 'FC Coins Trade on exchange', # Useful metadata to describe our partial state update blocks
        'policies': {
            'fc_purchase': p_buy_fc,
            'fc_sale': p_sell_fc,
        },
        'variables': {
            'fc_balance': s_fc_balance,
            'usd_balance': s_usd_balance,
        }
        
    },
     {
        'label': 'FC Price update', # Useful metadata to describe our partial state update blocks
        'policies': {
        },
        'variables': {
            'fc_price': s_fc_price
        }
        
    },
]


In [13]:
#MONTE_CARLO_RUNS = 50
SIMULATION_TIMESTEPS = 365

sim_config = config_sim(
    {
        'N': 1,
        'T': range(SIMULATION_TIMESTEPS),
        'M': system_params,
    }
)

from cadCAD import configs
del configs[:] # Clear any prior configs

experiment = Experiment()
experiment.append_configs(
    sim_configs=sim_config,
    initial_state=initial_state,
    partial_state_update_blocks=partial_state_update_blocks
)

In [14]:
exec_context = ExecutionContext()
run = Executor(exec_context=exec_context, configs=configs)

(system_events, tensor_field, sessions) = run.execute()


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (365, 1, 1, 5)
Execution Method: local_simulations
SimIDs   : [0]
SubsetIDs: [0]
Ns       : [0]
ExpIDs   : [0]
Execution Mode: single_threaded
Total execution time: 0.03s


In [16]:
df = pd.DataFrame(system_events)
df

,fc_minted,tree_planted,fc_price,usd_balance,fc_balance,simulation,subset,run,substep,timestep
0,6250.0,1000.0,0.500000,2500.000000,5000.000000,0,0,1,0,0
1,7325.0,1086.0,0.500000,2500.000000,5000.000000,0,0,1,1,1
2,7325.0,1086.0,0.500000,2574.544397,4850.911207,0,0,1,2,1
3,7325.0,1086.0,0.530734,2574.544397,4850.911207,0,0,1,3,1
4,7725.0,1118.0,0.530734,2574.544397,4850.911207,0,0,1,1,2
...,...,...,...,...,...,...,...,...,...,...
1091,243525.0,19982.0,2.884139,2458.841516,1209.966102,0,0,1,2,364
1092,243525.0,19982.0,2.032157,2458.841516,1209.966102,0,0,1,3,364
1093,243750.0,20000.0,2.032157,2458.841516,1209.966102,0,0,1,1,365
1094,243750.0,20000.0,2.032157,2733.269807,1074.923267,0,0,1,2,365


In [19]:
pd.options.plotting.backend = "plotly"
df.plot(kind='line', x='timestep', y=['tree_planted', 'fc_minted'])

In [20]:
df.plot(kind='line', x='timestep', y=['fc_price'])